In [1]:
import pandas as pd
import csv
import sys
import re
import scipy
import numpy as np
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint
from time import time

csv.field_size_limit(sys.maxsize)

131072

In [2]:
metrics = ['cbo','wmc','rfc','lcom','nom','nopm','nosm','nof','nopf','nosf','nosi','loc', "commits","linesAdded","linesDeleted","authors","minorAuthors","majorAuthors","authorOwnership"]

def get_metrics(row):
    features = []
    for metric in metrics:
        features.append(float(row[metric]))
    return features

In [3]:
df = pd.read_pickle('../data/instances.pkl')
labels = list(set(df['target'].values))
le = preprocessing.LabelEncoder()
le.fit(labels)

LabelEncoder()

In [4]:
X = []
Y = []

print("Preparing lists...")
for index, row in df.iterrows():
    X.append(get_metrics(row))
    Y.append(row["target"])

Preparing lists...


In [5]:
Y = le.transform(Y)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.75, random_state=42)

/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


# Default parameters

In [7]:
lr_classifier = LogisticRegression(random_state=42, verbose=1)
lr_classifier.fit(X_train, y_train)

/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]

/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=1, warm_start=False)

In [8]:
print("============ EVALUATION on test set:")
print(accuracy_score(y_test, lr_classifier.predict(X_test)))

============ EVALUATION on test set:
0.6459533191876327


In [9]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

param_dist = {'C': scipy.stats.expon(scale=100)}

lr_classifier2 = LogisticRegression(random_state=42, verbose=1)

n_iter_search = 20
random_search = RandomizedSearchCV(lr_classifier2,
                                   param_distributions=param_dist,
                                   n_iter=n_iter_search,
                                   cv=5,
                                   n_jobs=-1)
start = time()
print("Hyperparameter tuning...")
random_search.fit(X_train, y_train)
print("RandomizedSearchCV took %.2f seconds for %d candidates"
  " parameter settings." % ((time() - start), n_iter_search))
report(random_search.cv_results_)
print("============ EVALUATION on test set:")
print(accuracy_score(y_test, random_search.best_estimator_.predict(X_test)))

Hyperparameter tuning...


/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[LibLinear]RandomizedSearchCV took 603.14 seconds for 20 candidates parameter settings.
Model with rank: 1
Mean validation score: 0.654 (std: 0.004)
Parameters: {'C': 99.27391787530966}

Model with rank: 2
Mean validation score: 0.653 (std: 0.005)
Parameters: {'C': 392.7714661092794}

Model with rank: 3
Mean validation score: 0.653 (std: 0.003)
Parameters: {'C': 107.91983561975167}

============ EVALUATION on test set:
0.6450439527129433


/home/professor/.virtualenvs/refactoring_env/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
